In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#importing nltk and installing necessary libraries
import nltk
nltk.download('punkt')
nltk.download('stopwords')
!pip install emot
!pip install twarc
!pip install reverse_geocoder
!pip install contractions

converting csv file to txt file

In [ ]:
import pandas as pd
#reading the tweet id's column from csv file
dataframe=pd.read_csv("/content/drive/MyDrive/sempember_all.csv", header=None)

dataframe=dataframe[0]
#converting dataframe having tweet id's into text file
dataframe.to_csv("/content/drive/MyDrive/september_all.txt", index=False, header=None)

extracting Indian specific tweets from global dataset of tweets

In [ ]:
from twarc import Twarc
import reverse_geocoder as rg

file=open('/content/drive/My Drive/sept_tweet_id_NRT.txt',"w")
#authenticating credentials
consumer_key="NcAOmyCah64fNXfiiDoTefngv"
consumer_secret="deAqsF7kXvELzvptTAh4JuWgavgYZOfQle6BiTUVqiIlyPdsTq"
access_token="1312441901996367872-jB61aTImZI6lJ59jtylGtVbx0mrlhu"
access_token_secret="pry9rNZx3vTDr5Rr61Fa55V2AutMXi0l1iH4qhX87ni2F"

t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)
count=0
#code for hydrating tweets-text from tweet id's
for tweet in t.hydrate(open('/content/drive/MyDrive/New Text Document.txt')):
  #checking whether it's a Retweet
  if tweet['full_text'][0:2] != 'RT':
    user=tweet['user']
    place=tweet['place']
    #checking if the place is there(i.e.. not None) and it is of country 
    #India then we are storing the tweet in the file as we are considering 
    #only Indian specific tweets.
    if (place is not None) and (place['country_code']=='IN'):
      file.write(tweet['id_str']+"\n")
      count=count+1
      print(count)
    #If the place field is None but coordinates is known then we are trying to 
    #find country-code  using Longitude and Latitude of the tweet's place.
    elif tweet['coordinates']:
      coordinate=tweet['coordinates']['coordinates']
      result=rg.search(coordinate)
      if result[0]['cc']=='IN':
        file.write(tweet['id_str']+"\n")
        count=count+1
        print(count)
    #if cordinates is also not there then we will try to get tweets from location field
    elif 'india' in (user['location']).lower():
      file.write(tweet['id_str']+"\n")
      count=count+1
      print(count)


preparing training dataset such that for each tweet we have it's tweet id, tweet text and label. 

In [ ]:
from twarc import Twarc
from textblob import TextBlob

consumer_key="NcAOmyCah64fNXfiiDoTefngv"
consumer_secret="deAqsF7kXvELzvptTAh4JuWgavgYZOfQle6BiTUVqiIlyPdsTq"
access_token="1312441901996367872-jB61aTImZI6lJ59jtylGtVbx0mrlhu"
access_token_secret="pry9rNZx3vTDr5Rr61Fa55V2AutMXi0l1iH4qhX87ni2F"

t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

trainingDataset=[]

for tweet in t.hydrate(open('/content/drive/MyDrive/training.txt')):
  t={}
  t['tweet_id']=tweet['id']
  t['text']=tweet['full_text']
  polarity=TextBlob(t['text']).sentiment.polarity
  if polarity > 0:
    t['label']='positive'
  elif polarity <0:
    t['label']='negative'
  else:
    t['label']='neutral'
  trainingDataset.append(t)


preparing evaluation dataset same format as that of training dataset

In [ ]:
from twarc import Twarc
from textblob import TextBlob
consumer_key="NcAOmyCah64fNXfiiDoTefngv"

consumer_secret="deAqsF7kXvELzvptTAh4JuWgavgYZOfQle6BiTUVqiIlyPdsTq"

access_token="1312441901996367872-jB61aTImZI6lJ59jtylGtVbx0mrlhu"

access_token_secret="pry9rNZx3vTDr5Rr61Fa55V2AutMXi0l1iH4qhX87ni2F"

t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)

evaluationDataset=[]

for tweet in t.hydrate(open('/content/drive/MyDrive/evaluation.txt')):
  t={}
  t['tweet_id']=tweet['id']
  t['text']=tweet['full_text']
  polarity=TextBlob(t['text']).sentiment.polarity
  if polarity > 0:
    t['label']='positive'
  elif polarity <0:
    t['label']='negative'
  else:
    t['label']='neutral'
  evaluationDataset.append(t)


preparing test dataset such that for each tweet we have it's tweet id, tweet text and label. where label is initialized with None value.
we have two dataset here one for the month of April and another one for September month.

In [ ]:
from twarc import Twarc
from textblob import TextBlob
consumer_key="NcAOmyCah64fNXfiiDoTefngv"

consumer_secret="deAqsF7kXvELzvptTAh4JuWgavgYZOfQle6BiTUVqiIlyPdsTq"

access_token="1312441901996367872-jB61aTImZI6lJ59jtylGtVbx0mrlhu"

access_token_secret="pry9rNZx3vTDr5Rr61Fa55V2AutMXi0l1iH4qhX87ni2F"

t = Twarc(consumer_key, consumer_secret, access_token, access_token_secret)
testDataset_apr=[]

for tweet in t.hydrate(open('/content/drive/MyDrive/test_april.txt')):
  t={}
  t['tweet_id']=tweet['id']
  t['text']=tweet['full_text']
  t['label']=None
  testDataset_apr.append(t)
testDataset_sep=[]
for tweet in t.hydrate(open('/content/drive/MyDrive/test_sept.csv')):
  t={}
  t['tweet_id']=tweet['id']
  t['text']=tweet['full_text']
  t['label']=None
  testDataset_sep.append(t)


In [ ]:
print(len(trainingDataset))
print(len(evaluationDataset))

code for preprocessing of tweets

In [ ]:
import contractions 
from nltk.tokenize import word_tokenize
import string 
from string import punctuation
from nltk.corpus import stopwords
import re
from emot.emo_unicode import UNICODE_EMO, EMOTICONS# Function for removing emoticons


class PreProcessTweets:
    _stopwords=[]
    def _init_(self):
        self._stopwords = set(stopwords.words('english') + list(punctuation) + ['ATUSER','URL'])
        
    def processTweets(self, list_of_tweets):
        processedTweets=[]
        for tweet in list_of_tweets:
            processedTweets.append((self._processTweet(tweet["text"]),tweet["label"]))
        return processedTweets
    
    def _processTweet(self, tweet):
        tweet = tweet.lower() # convert text to lower-case
        tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
        tweet = re.sub('@[^\s]+', 'ATUSER', tweet) # remove usernames
        tweet = re.sub(r'#([^\s]+)', r'\1', tweet) # remove the # in #hashtag
        tweet = re.sub(r'(.)\1+', r'\1\1', tweet)  #remove the repeating characters
        for emot in EMOTICONS:
          tweet = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), tweet) 
        expanded_words = []	
        for word in tweet.split(): 
          expanded_words.append(contractions.fix(word));
        punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
        str1_list =[]
        for word1 in expanded_words:
          str1 ="";
          flag =0;
          for each_char in word1: 
            if each_char not in punctuations:
              str1 = str1 + each_char
              flag =1
          if flag == 1:
            str1_list.append(str1)
        return [word for word in str1_list if word not in self._stopwords]

In [ ]:
print(stopwords.words('english'))

In [ ]:
tweetsProcessor=PreProcessTweets()
PPTrainingset=tweetsProcessor.processTweets(trainingDataset)
PPEvaluationset=tweetsProcessor.processTweets(evaluationDataset)
PPTestset_apr=tweetsProcessor.processTweets(testDataset_apr)
PPTestset_sep=tweetsProcessor.processTweets(testDataset_sep)

In [ ]:
print(len(PPTrainingset))
print(len(PPTestset))

creating word feature such that it has all the unique words from our training dataset

In [ ]:
from nltk import FreqDist
tokens=[]
word_feature=[]
for (words,label) in PPTrainingset:
  tokens.extend(words)
freq_dist_tokens=FreqDist(tokens)
word_feature=freq_dist_tokens.keys()

In [ ]:
print(len(word_feature))
print(len(tokens))

48762
777686


function which takes input as a particular tweet and returns feature vector for  that tweet. feature vector is designed using one hot encoding technique.

In [ ]:
def extract_features(tweet):
  tweet_words = set(tweet)
  features = {}
  for word in word_feature:
    features['contains(%s)' % word] = (word in tweet_words)
  return features 

with the help of apply_features method of NLTK all the feature vectors of all the tweets in training set all stored space efficiently in trainingFeatures object.

In [ ]:
import nltk
trainingFeatures = nltk.classify.apply_features(extract_features, PPTrainingset)

training the NBayes Classifier model.

In [ ]:
NBayesClassifier=nltk.NaiveBayesClassifier.train(trainingFeatures)

In [ ]:
#Finding the accuracy of the model with tagged data.
count=0
for (tweet,label) in PPEvaluationset:
  label==NBayesClassifier.classify(extract_features(tweet)):
    count+=1
Accuracy=(float(count)/len(PPEvaluationset))*100
print(Accuracy)

71.9978914074855


In [ ]:
pos_count=0
neg_count=0
neu_count=0
for (tweet,label) in PPtestset_apr:
  label=NBayesClassifier.classify(extract_features(tweet))
  if label=='positive':
    pos_count+=1
  elif label=='negative':
    neg_count+=1
  else:
    neu_count+=1


In [ ]:
import matplotlib.pyplot as plt 

activities = ['positive', 'negative','neutral']
  
# portion covered by each label
slices = [pos_count,neg_count,neu_count]
  
# color for each label
colors = ['g', 'r','b']
  
# plotting the pie chart
plt.pie(slices, labels = activities, colors=colors, 
        startangle=90, shadow = True, 
        radius = 1.2, autopct = '%1.1f%%')
  
# plotting legend
plt.legend()
  
# showing the plot
plt.show()

In [ ]:
pos_count=0
neg_count=0
neu_count=0
for (tweet,label) in PPtestset_sep:
  label=NBayesClassifier.classify(extract_features(tweet))
  if label=='positive':
    pos_count+=1
  elif label=='negative':
    neg_count+=1
  else:
    neu_count+=1


In [ ]:
import matplotlib.pyplot as plt 

activities = ['positive', 'negative','neutral']
  
# portion covered by each label
slices = [pos_count,neg_count,neu_count]
  
# color for each label
colors = ['g', 'r','b']
  
# plotting the pie chart
plt.pie(slices, labels = activities, colors=colors, 
        startangle=90, shadow = True, 
        radius = 1.2, autopct = '%1.1f%%')
  
# plotting legend
plt.legend()
  
# showing the plot
plt.show()